# ABOUT


Datascientest's Datascientist continuous bootcamp - cohorte Mars2022 -  AeroBOT project

**Tutor**

* Alban THUET

**Authors:**

* Hélène ASSIR
* Hichem HADJI  
* [Ioannis STASINOPOULOS](https://www.linkedin.com/in/ioannis-stasinopoulos/)

</br>

---
</br>

**Version History**

Version | Date       | Author(s)  | Modification
--------|----------- | ---------  | --------------------------
X.X     | XX/XX/2022 | X.X        | modif
1.0     | 21/07/2022 | I.S, H.A.  | Document creation

This notebook 


*   loads the raw data donwloaded from the ASRS in to a pandas DataFrame (total of `108407` entries)
*   removes 2 columns named 'Unnammed: ...' that originate from the concatenation of several .csv files into the `ASRS_20y_data.csv` file
* removes the 166 data related to UAS
* removes the UAS-related columns (features) 
* allocates 10% (i.e. `10824` entries) of the total number of entries to a test set. *In the test set only entries after 2010 are present!*
Why? This test set emulates new data that will arrive after 2022. Helene observed that Narratives were written in uppercase letters before 2010.
The resulting train set contains `97417` entries and 96 columns
* saves the train and test sets as .pkl files, named `train_data_final.pkl `and `test_data_final.pkl`, respectively. 
Further processing of the data should be perfomed on the `train_data_final.pkl ` data.

# IMPORT PACKAGES


In [ ]:
#######################
# Import packages
#######################
import numpy as np
import seaborn as sns

import pandas as pd
# Set pandas settings to show all data when using .head(), .columns etc.
pd.options.display.max_columns = None
pd.options.display.max_rows = None

import itertools # Pour créer des iterateurs

######################
# PLOTTING
######################
import matplotlib.pyplot as plt
%matplotlib inline
# Define global plot parameters for better readability and consistency among plots
# A complete list of the rcParams keys can be retrieved via plt.rcParams.keys() function
plt.rcParams['axes.titlesize'] = 30
plt.rcParams['axes.labelsize'] = 23
plt.rcParams['xtick.labelsize'] = 23
plt.rcParams['ytick.labelsize'] = 23
plt.rc('legend', fontsize=23)    # legend fontsize

# BOKEH 
from bokeh.plotting import figure # Importation de la classe figure qui permet de créer un graphique bokeh.
from bokeh.io import  push_notebook, output_notebook, show
output_notebook() # permet d'afficher tous les futurs graphiques dans l'output d'une cellule jupyter. Si cette instruction n'est pas lancée, la figure s'affichera dans un nouvel onglet.
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.models.tools import HoverTool

#####################
# NLP 
#####################
import re # for Regular Expression handling
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # WordNet lemmatizer
nltk.download('omw-1.4') # necessary for WordNet lemmatizer
from nltk.tokenize import word_tokenize # Usual tokenizer
from nltk.tokenize import TweetTokenizer # Special tokenizer;  "we'll", "didn't", etc. are considered as one word
from sklearn.feature_extraction.text import CountVectorizer # Vectorization
from nltk.corpus import stopwords # Import stopwords from nltk.corpus

###############################
# ML preprocessing and models
###############################
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report, confusion_matrix

import pickle as pkl # Saving data externally

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# LOAD DATA

## Mount GDrive

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive/')

#check your present working directory 
%pwd

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


'/content/drive/My Drive/data'

In [ ]:
# move to the desired location (adapt to your folder-tree-structure):
%cd /content/drive/MyDrive/data/

/content/drive/MyDrive/data


In [ ]:
!ls # list the content of the pwd

#!ls "/content/drive/MyDrive/Data_Science/Formations/DataScienceTest/projet/AeroBot/" # list contect of a speficic folder

 ASRS_20y_data.csv	        State_Reference_Dictionnary.csv
'Data Dictionnary themes.csv'


## Load the ASRS_20y_data.csv data




In [ ]:
df = pd.read_csv('ASRS_20y_data.csv', low_memory=False,index_col=1)
# See https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options

In [ ]:
print("A total of", len(df), "entries have been loaded.")

A total of 108407 entries have been loaded.


In [ ]:
df.columns

Index(['Unnamed: 0', 'Date', 'Local Time Of Day', 'Locale Reference',
       'State Reference', 'Relative Position.Angle.Radial',
       'Relative Position.Distance.Nautical Miles',
       'Altitude.AGL.Single Value', 'Altitude.MSL.Single Value',
       'Latitude / Longitude (UAS)',
       ...
       'When Detected', 'Result', 'Contributing Factors / Situations',
       'Primary Problem', 'Narrative', 'Callback', 'Narrative.1', 'Callback.1',
       'Synopsis', 'Unnamed: 125'],
      dtype='object', length=126)

In [ ]:
df.shape

(108407, 126)

## Remove empty columns

In [ ]:
# Remove Unnamed columns
df = df.drop(['Unnamed: 0','Unnamed: 125'], axis = 1)    
df.shape

NameError: ignored

## Remove UAS entries

In [ ]:
UAS_column_list=[]
for col in df.columns:
  if 'UAS' in col :
    UAS_column_list.append(col)

In [ ]:
UAS_column_list

['Latitude / Longitude (UAS)',
 'Airspace Authorization Provider (UAS)',
 'Operating Under Waivers / Exemptions / Authorizations (UAS)',
 'Waivers / Exemptions / Authorizations (UAS)',
 'Airworthiness Certification (UAS)',
 'Weight Category (UAS)',
 'Configuration (UAS)',
 'Flight Operated As (UAS)',
 'Flight Operated with Visual Observer (UAS)',
 'Control Mode (UAS)',
 'Flying In / Near / Over (UAS)',
 'Passenger Capable (UAS)',
 'Type (UAS)',
 'Number of UAS Being Controlled (UAS)',
 'Airspace Authorization Provider (UAS).1',
 'Operating Under Waivers / Exemptions / Authorizations (UAS).1',
 'Waivers / Exemptions / Authorizations (UAS).1',
 'Airworthiness Certification (UAS).1',
 'Weight Category (UAS).1',
 'Configuration (UAS).1',
 'Flight Operated As (UAS).1',
 'Flight Operated with Visual Observer (UAS).1',
 'Control Mode (UAS).1',
 'Flying In / Near / Over (UAS).1',
 'Passenger Capable (UAS).1',
 'Type (UAS).1',
 'Number of UAS Being Controlled (UAS).1',
 'UAS Communication Break

In [ ]:
# Remove entries with non empty UAS columns
for col_UAS in  UAS_column_list:
    df = df[df[col_UAS].isnull()]
df.shape

(108241, 124)

In [ ]:
# Remove UAS columns
df = df.drop(UAS_column_list, axis = 1)    
df.shape

(108241, 95)

**Yannis checks one RegEx**

---





## Split into TRAIN, VALIDATION, TEST sets



*   the test set emulates the future data that we will get. It is set completely aside



### Create 'Year' feature

In [ ]:
df['Year']=df.Date.astype(str).apply(lambda x :  x if  len(x)==4 else x[:-2] ).astype(int) 

In [ ]:
# Sub-df containing entries from years 2010 (to early 2022)
df_recent = df[df['Year'] >= 2010]

my_test_size = int(np.round(0.1 * len(df), 0))
print("We have a total of", len(df), 'entries.')
print("We allocate 10% of the total number of entries to the test set; this amounts to:", my_test_size, 'entries.')

df_recent_train, df_test = train_test_split(df_recent ,test_size=my_test_size,random_state=1234)
print("We have ", len(df_test), 'entries in our test set')

df_train=pd.concat ([df_recent_train, df[df['Year'] < 2010]]) 
print("We have ", len(df_train), 'entries in our train set')
df_train.shape

We have a total of 108241 entries.
We allocate 10% of the total number of entries to the test set; this amounts to: 10824 entries.
We have  10824 entries in our test set
We have  97417 entries in our train set


(97417, 96)

## Save / Load datasets


In [ ]:
##########################################################
# WARNING!! 
# If you execute this cell, you will OVERWRITTE the data!
##########################################################

# %cd /content/drive/MyDrive/data/transformed/

# #to save the data
# # Save TEST data
# with open("test_data_final.pkl", "wb") as f:
#     pkl.dump([df_test], f) # saves the variables into a list

# # Save TRAIN data
# with open("train_data_final.pkl", "wb") as f:
#     pkl.dump([df_train], f) # saves the variables into a list

/content/drive/MyDrive/data/transformed
